# Delta Exploration Notebook

## What is Delta $(\Delta)$?

Delta measures how much an option price contract changes for every change in $\$1.00$ of the underlying security. This makes it the slope of the option price curve.

## Example

$\Delta = 0.6$

When the underlying stock goes up $\$1$ the option only gains $\$0.60$ in value.

In [27]:
import random
import math

In [28]:
stock_price = 100
strike_price = 100
option_price = 50

For simplicity sake we will have the stock price change by some random amount and have the option value change by some random amount.

In [29]:
stock_price_change = random.uniform(-3, 3)
option_price_change = random.uniform(-2, 2)

new_stock_price = stock_price + stock_price_change
new_option_price = option_price + option_price_change

print('New Stock Price: ' + str(round(new_stock_price, 4)))
print('New Option Price: ' + str(round(new_option_price, 4)))

New Stock Price: 97.4949
New Option Price: 50.1726


## Formula

Now we can reverse engineer the delta by finding the difference in the new and old prices of the underlying secuirty and the option price.

**Formal Definition**

$$\Delta = \frac{\partial C}{\partial S}$$

**Practical Definition**

$$\Delta = \frac{\text{change in option price}}{\text{change in stock price}}$$

In [30]:
def calculate_delta(s1, s2, o1, o2):
    """
    Calculate delta as the ratio of option price change to stock price change.
    
    Args:
        s1: Initial stock price
        s2: Final stock price
        o1: Initial option price
        o2: Final option price
    
    Returns:
        The calculated delta value
    """
    stock_change = s2 - s1
    option_change = o2 - o1

    return option_change / stock_change

In [31]:
implied_delta = calculate_delta(
    s1=stock_price, 
    s2=new_stock_price,
     
    o1=option_price, 
    o2=new_option_price
)

print('Delta = ' + str(round(implied_delta, 4)))

Delta = -0.0689


Due to the random way I generated option price changes, we can get $|\Delta| > 1$, which is impossible in the real world. For a **call option**, delta must be between 0 and 1 because:

- The option can never be worth more than the underlying stock
- The option price can't change by more than the stock price change

To fix this, I'll model a **call option** properly where:
- The option and stock move in the **same direction** 
- The option change is 80% of the stock change
- This should eventually give us a realistic delta of 0.8

In [32]:
# Clean up variables
if stock_price_change >= 0:
    option_price_change = abs(stock_price_change) * 0.80
else:
    option_price_change = stock_price_change * 0.8
    
new_option_price = option_price + option_price_change

print('Option Price Change: ' + str(round(option_price_change, 4)))

Option Price Change: -2.004


In [33]:
print('New Stock Price: ' + str(round(new_stock_price, 4)))
print('New Option Price: ' + str(round(new_option_price, 4)))

New Stock Price: 97.4949
New Option Price: 47.996


In [34]:
# calculate delta again with the new values

implied_delta = calculate_delta(
    s1=stock_price, 
    s2=new_stock_price, 
    
    o1=option_price, 
    o2=new_option_price
)
print('Delta = ' + str(round(implied_delta, 4)))

Delta = 0.8


Now here we should see that $|\Delta| = 0.8$ because the underlying option price always changes $\$0.8$ for every $\$1.0$ the stock price moves

In [35]:
print("\n=== MULTIPLE DELTA TESTS ===")

deltas = []

for i in range(10):
    stock_price_change = random.uniform(-3, 3)
    
    if stock_price_change >= 0:
        option_price_change = abs(stock_price_change) * 0.80
    else:
        option_price_change = stock_price_change * 0.8
    
    new_stock_price = stock_price + stock_price_change
    new_option_price = option_price + option_price_change
    
    delta = calculate_delta(stock_price, new_stock_price, option_price, new_option_price)
    deltas.append(delta)
    
    print(f"Test {i+1}: Stock ${stock_price_change:+.2f} → Delta = {delta:.3f}")


=== MULTIPLE DELTA TESTS ===
Test 1: Stock $+1.63 → Delta = 0.800
Test 2: Stock $-2.61 → Delta = 0.800
Test 3: Stock $+0.43 → Delta = 0.800
Test 4: Stock $-0.33 → Delta = 0.800
Test 5: Stock $-0.77 → Delta = 0.800
Test 6: Stock $-1.39 → Delta = 0.800
Test 7: Stock $-1.34 → Delta = 0.800
Test 8: Stock $-1.82 → Delta = 0.800
Test 9: Stock $+1.45 → Delta = 0.800
Test 10: Stock $-1.06 → Delta = 0.800


In [36]:
print("\n=== DELTA VALIDATION ===")
print(f"All deltas between 0 and 1? {all(0 <= d <= 1 for d in deltas)}")
print(f"Average delta: {sum(deltas)/len(deltas):.3f}")


=== DELTA VALIDATION ===
All deltas between 0 and 1? True
Average delta: 0.800
